In [2]:
import numpy as np
import pickle

In [5]:
# Storing the 10k model
long_model = np.load('../agent_code/agent_vq10/logs/Q_data/Q9483.npy')
with open('../agent_code/agent_vq10/other_models/model_vq10_10k.pt', "wb") as file:
        pickle.dump(long_model, file)